In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 659.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.8/871.8 kB 5.8 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

MODEL_PATH = '/content/yolov8n.pt'

model = YOLO(MODEL_PATH)

In [ ]:
model

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [ ]:
def get_person(class_names_dict, target_class='person'):
    for key, val in class_names_dict.items():
        if val == target_class:
            return key

In [ ]:
import cv2
import numpy as np

In [ ]:
VIDEO_LOC = '/content/WhatsApp Video 2024-09-08 at 22.57.20_5e6ce80c.mp4'

In [ ]:
x1, y1, x2, y2 = (500, 0, 848, 478)

In [ ]:
video_capture = cv2.VideoCapture(VIDEO_LOC)

In [ ]:
# Function to get the person_id from the classes identifiable by the model
def get_person(class_names_dict, target_class='person'):
    for key, val in class_names_dict.items():
        if val == target_class:
            return key

import cv2
import numpy as np
from google.colab.patches import cv2_imshow

VIDEO_LOC = '/content/WhatsApp Video 2024-09-08 at 22.57.20_5e6ce80c.mp4'

# 1. pixel select
x1, y1, x2, y2 = (500, 0, 848, 478)

# 2. video
video_capture = cv2.VideoCapture(VIDEO_LOC)

while video_capture.isOpened():
    success, frame = video_capture.read()

    if not success or frame is None:
        print("Failed to read frame or frame is None.")
        continue

    # Proceed if the frame is valid
    try:
        # 3. read frame
        result_set = model(frame)[0]

        # 4. find person
        person_id = get_person(result_set.names)
        # 5. filter person/get pixel for restriction
        person_img_masked = [res for res in result_set if res.boxes.cls == person_id]
        if len(person_img_masked) < 1:
            # 5.A. if person not found
            continue
        else:
            # 5.B.
            person_img_masked = person_img_masked[0]
        # 5.B.
        person_region = person_img_masked.boxes.xyxy[0].numpy()

        # 6. Highlight Area
        annotated_frame = person_img_masked.plot()

        region_img = annotated_frame[y1:y2, x1:x2]
        region = np.zeros(region_img.shape, dtype=np.uint8)
        res = cv2.addWeighted(region, 0.7, region_img, 0.5, 1.0)
        annotated_frame[y1:y2, x1:x2] = res

        # 7. Calculate Overlap
        masked_img_map = np.zeros(annotated_frame.shape)
        masked_img_map[int(person_region[1]):int(person_region[3]), int(person_region[0]):int(person_region[2])] = 1
        overlap_reg_map = np.zeros(annotated_frame.shape)
        overlap_reg_map[y1:y2, x1:x2] = 1

        union = masked_img_map + overlap_reg_map
        overlap = masked_img_map * overlap_reg_map
        IOU = overlap.sum()/float(union.sum())

        # 9. Alert Message
        if IOU > 0.1:
            cv2.putText(annotated_frame, 'PERSON INSIDE RESTRICTED AREA', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        frame = annotated_frame

        # Display the resulting frame
        cv2_imshow(frame)
    except Exception as e:
        print(f"Error during frame processing: {e}")
        break

